# Obteniendo el TOTAL de casos positivos, fallecidos y vacunados contra COVID-19 por cada departamento del Perú

En este notebook se busca obtener un dataset con el TOTAL con el número de **casos positivos**, **fallecidos** y **totalmente vacunados** por cada **departamento** del Perú. De la misma manera se agrega el **número de habitantes** de cada departamento, para calcular la **tasa de mortalidad por 100 000 habitantes** y el **porcentaje de vacunación** respectivo.

Se considera como **completamente vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

En esta sección se lee y procesa el dataset **Data Product 1** ([DP1_covid19-peru_x_semanaEpi.csv](Data/DP1_covid19-peru_x_semanaEpi.csv)) para obtener un resumen general por departamento.

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

In [1]:
import pandas as pd
import dask.dataframe as dd
import functions as fn

In [2]:
# Cargamos un diccionario con el número de habitantes por departamento en 2020
dic_dep = {
    "AMAZONAS"	:   426806,
    "ANCASH"	:   1180638,
    "APURIMAC"	:   430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:   668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
}

# Convertimos a dataframe
df_dep = pd.DataFrame(dic_dep.items(), columns = ['departamento', 'no_habitantes'])

In [3]:
# Leer el DP1
url_dp1 = '../Data/DP1_covid19-peru_x_semanaEpi.csv'
df = pd.read_csv(url_dp1)

# Procesar el df (sumar cada col)
df = pd.DataFrame(df.sum(axis = 0, skipna = True)).reset_index().rename({'index': 'var', 0: 'total'}, axis=1)

# Dejar solamente filas que contengan el nombre de un departamento
lst_dfs = [] 

for dep in df_dep['departamento']:
    temp_df = df.loc[df['var'].str.contains(dep)]
    if not temp_df.empty:
        lst_dfs.append(temp_df)
        
df = pd.concat(lst_dfs, ignore_index=True).drop_duplicates()  # Drop duplicates

# Eliminamos a vacunados (_vac) porque necesitamos todas las dosis 1, 2 y 3 no solo vacunación completa
df = df[~df['var'].str.contains('_vac')]  
    
# Separate in sub dfs and remove substrings
df_cas = df[df['var'].str.contains('_cas')].rename({'var': 'departamento','total': 'casos'}, axis = 1)
df_cas['departamento'] = df_cas['departamento'].str.replace('_cas', '')
df_fal = df[df['var'].str.contains('_fal')].rename({'var': 'departamento','total': 'fallecidos'}, axis = 1)
df_fal['departamento'] = df_fal['departamento'].str.replace('_fal', '')

# Unir en un solo dataframe el resumen por departamentos 
df_summary = df_dep.merge(df_cas, how='left', on='departamento')
df_summary = df_summary.merge(df_fal, how='left', on='departamento')

df_summary

,departamento,no_habitantes,casos,fallecidos
0,AMAZONAS,426806,43741,1332
1,ANCASH,1180638,124289,7068
2,APURIMAC,430736,38638,1618
3,AREQUIPA,1497438,211718,10208
4,AYACUCHO,668213,47880,2306
5,CAJAMARCA,1453711,95462,4425
6,CALLAO,1129854,146106,10458
7,CUSCO,1357075,116277,5121
8,HUANCAVELICA,365317,25529,1257
9,HUANUCO,760267,51767,2859


## B. Agregar el número de dosis

In [4]:
# Leer solamente "id_eess" y "dosis" del dataset de vacunados
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
vac_col = ['id_eess', 'dosis']
df_vac = pd.read_csv(url_vac, sep = ",", usecols = vac_col, dtype = {'id_eess':'int32','dosis': 'int8'})

In [5]:
df_vac

,dosis,id_eess
0,2,3441
1,3,24541
2,1,2522
3,2,12771
4,1,4320
...,...,...
63386860,1,1679
63386861,1,5206
63386862,2,1701
63386863,2,8755


In [6]:
vac_doses = pd.get_dummies(df_vac['dosis']).drop([4, 5], axis = 1)

In [7]:
vac_doses

,1,2,3
0,0,1,0
1,0,0,1
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
63386860,1,0,0
63386861,1,0,0
63386862,0,1,0
63386863,0,1,0


In [8]:
dosis = pd.concat([df_vac, vac_doses], axis=1).drop('dosis', axis=1)
del df_vac, vac_doses

In [9]:
vac_sum = dosis.groupby(['id_eess']).sum().reset_index()
del dosis

In [16]:
vac_sum

,id_eess,1,2,3
0,1,1197.0,1198.0,1313.0
1,3,1848.0,1621.0,1202.0
2,4,932.0,713.0,408.0
3,5,386.0,348.0,152.0
4,6,770.0,524.0,124.0
...,...,...,...,...
8079,29089,229.0,226.0,0.0
8080,29111,0.0,7.0,6.0
8081,29124,8.0,8.0,4.0
8082,29144,21.0,21.0,0.0


In [12]:
# Leer los datasets de ubigeo y centros de vacunación
url_ubigeo = '../RawData/TB_UBIGEOS.csv'
url_vaccenter = '../RawData/TB_CENTRO_VACUNACION.csv'
ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante "id_ubigeo"
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left').sort_values(by='id_eess')
del ubigeo, vaccenter['id_ubigeo']

In [13]:
vaccenter

,id_eess,departamento
0,0,LIMA
31076,0,AYACUCHO
31077,0,CAJAMARCA
31078,0,HUANCAVELICA
31079,0,HUANCAVELICA
...,...,...
30373,28382,PUNO
30374,28383,PUNO
30375,28384,PUNO
30376,28385,CALLAO


In [17]:
# Encontrar el departamento correspondiente de todas las personas vacunadas
vacDep = vac_sum.merge(vaccenter, on = 'id_eess', how = 'left')


In [22]:
vacDep

,id_eess,1,2,3,departamento
0,1,1197.0,1198.0,1313.0,LORETO
1,3,1848.0,1621.0,1202.0,LORETO
2,4,932.0,713.0,408.0,LORETO
3,5,386.0,348.0,152.0,LORETO
4,6,770.0,524.0,124.0,LORETO
...,...,...,...,...,...
8079,29089,229.0,226.0,0.0,NaN
8080,29111,0.0,7.0,6.0,NaN
8081,29124,8.0,8.0,4.0,NaN
8082,29144,21.0,21.0,0.0,NaN


In [23]:

# Contar el total de número de dosis por departamento
df_vac = vacDep.groupby('departamento').agg({1: ['sum'], 2: ['sum'], 3: ['sum']})


In [24]:
df_vac

,1,2,3
,sum,sum,sum
departamento,,,
AMAZONAS,308345.0,261416.0,98876.0
ANCASH,1037163.0,952289.0,472763.0
APURIMAC,387570.0,332894.0,125542.0
AREQUIPA,1283375.0,1154332.0,476804.0
AYACUCHO,485048.0,400117.0,111305.0
CAJAMARCA,1153665.0,994226.0,326159.0
CALLAO,1019715.0,947901.0,479305.0
CUSCO,1041344.0,900482.0,280303.0


In [26]:
df_vac = df_vac.astype({(1, 'sum'): 'int64', (2, 'sum'): 'int64', (3, 'sum'): 'int64'})

In [27]:
# Unir al df resumen (df_summary)
df_summary = df_summary.merge(df_vac, how='left', on='departamento')

# Rename cols
df_summary.rename(columns = {'index':'departamentos', (1, 'sum'):'dosis_1', 
                             (2, 'sum'): 'dosis_2', (3, 'sum'):'dosis_3'}, inplace = True)


/home/xxotto/anaconda3/envs/emergelab/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


In [28]:
df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3
0,AMAZONAS,426806,43741,1332,308345,261416,98876
1,ANCASH,1180638,124289,7068,1037163,952289,472763
2,APURIMAC,430736,38638,1618,387570,332894,125542
3,AREQUIPA,1497438,211718,10208,1283375,1154332,476804
4,AYACUCHO,668213,47880,2306,485048,400117,111305
5,CAJAMARCA,1453711,95462,4425,1153665,994226,326159
6,CALLAO,1129854,146106,10458,1019715,947901,479305
7,CUSCO,1357075,116277,5121,1041344,900482,280303
8,HUANCAVELICA,365317,25529,1257,279753,220726,59320
9,HUANUCO,760267,51767,2859,497744,416077,139691


## C. Añadir detalles

In [29]:
# Mortality rate per 100k per department
df_summary['tasa_mortalidad'] = round((df_summary['fallecidos']/df_summary['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
df_summary['vac_porcentaje'] = round((df_summary['dosis_2']*100)/df_summary['no_habitantes'],2)

In [30]:
df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3,tasa_mortalidad,vac_porcentaje
0,AMAZONAS,426806,43741,1332,308345,261416,98876,312.09,61.25
1,ANCASH,1180638,124289,7068,1037163,952289,472763,598.66,80.66
2,APURIMAC,430736,38638,1618,387570,332894,125542,375.64,77.28
3,AREQUIPA,1497438,211718,10208,1283375,1154332,476804,681.70,77.09
4,AYACUCHO,668213,47880,2306,485048,400117,111305,345.10,59.88
5,CAJAMARCA,1453711,95462,4425,1153665,994226,326159,304.39,68.39
6,CALLAO,1129854,146106,10458,1019715,947901,479305,925.61,83.90
7,CUSCO,1357075,116277,5121,1041344,900482,280303,377.36,66.35
8,HUANCAVELICA,365317,25529,1257,279753,220726,59320,344.08,60.42
9,HUANUCO,760267,51767,2859,497744,416077,139691,376.05,54.73


In [31]:
# Resumen general
print(f'Total habitantes: {df_summary.no_habitantes.sum()}')
print(f'Total fallecidos: {df_summary.fallecidos.sum()}')
print(f'Total vacunados con 1 dosis: {df_summary.dosis_1.sum()}')
print(f'Total vacunados con 2 dosis: {df_summary.dosis_2.sum()}')
print(f'Total vacunados con 3 dosis: {df_summary.dosis_3.sum()}')
print(f'Porcentaje de la población vacunada: {round(df_summary.dosis_2.sum()*100/df_summary.no_habitantes.sum(),2)}')

Total habitantes: 32625948
Total fallecidos: 211423
Total vacunados con 1 dosis: 27947864
Total vacunados con 2 dosis: 24788890
Total vacunados con 3 dosis: 10646339
Porcentaje de la población vacunada: 75.98


## D. Guardar el dataset

In [32]:
df_summary.to_csv('../Data/DP2_covid19-peru_resumen_x_departamentos.csv',index = False)